In [76]:
import pandas as pd
import re

In [82]:
from bs4 import BeautifulSoup
# Funktsioon html-vormingute eemaldamiseks
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [103]:
#konflikt = 'remmelga'
konflikt = 'tselluloositehase'

# Esialgne tsv-formaadis fail on konverteeritud xlsx-formaati, lisatud esimene rida tekstiväljade pealkirjadega
data = pd.read_excel(konflikt + '_tekstid.xlsx', engine='openpyxl')

nimed_list = data.nimi.to_list()

# Kuupäeva eraldamine
# Väljaande eraldamine
kuupaevad = []
valjaanded = []
for nimi in nimed_list:
    match1 = re.search(r'[a-z]+', nimi)
    match2 = re.search(r'\d+', nimi) 
    if match1:
        valjaanne = match1[0]
    if match2:
        aasta = match2[0][0:4]
        kuu = match2[0][4:6]
        paev = match2[0][6:8]
        kuupaev = aasta +'-'+ kuu + '-'+paev
    kuupaevad.append(kuupaev)
    valjaanded.append(valjaanne)

# Teksti puhastamine html-i vormingutest
tekst_list = data['text_must'].to_list()
tekst_puhas = []
for tekst in tekst_list:
    s = strip_html_tags(tekst)    
    tekst_puhas.append(s)
    
# Tabelisse vastavate veergude lisamine, ebavajaliku eemaldamine
del data['text_must']
del data['nimi']

data.insert(0,'text', tekst_puhas)
data.insert(2,'subject', valjaanded)
data.insert(3,'date', kuupaevad)

data.to_csv(konflikt + '_tekstid_puhastatud.csv',index=False)
read_file = pd.read_csv (konflikt + '_tekstid_puhastatud.csv')
read_file.to_excel (konflikt + '_tekstid_puhastatud.xlsx', index = None, header=True)